In [2]:
import pandas as pd
import numpy as np
import os
import spacy


import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
ds = tfds.load('scientific_papers', split='train', shuffle_files=True)

2023-05-30 11:29:19.719326: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

: 

: 

In [ ]:
assert isinstance(ds, tf.data.Dataset)

In [ ]:
print(ds)

In [3]:
df = pd.read_csv('./data/dataset.csv')

In [4]:
df.head()

,Text,Summary,Class
0,Musicians to tackle US red tape\n\nMusicians' ...,Nigel McCune from the Musicians' Union said Br...,entertainment
1,"U2's desire to be number one\n\nU2, who have w...",But they still want more.They have to want to ...,entertainment
2,Rocker Doherty in on-stage fight\n\nRock singe...,"Babyshambles, which he formed after his acrimo...",entertainment
3,Snicket tops US box office chart\n\nThe film a...,A Series of Unfortunate Events also stars Scot...,entertainment
4,Ocean's Twelve raids box office\n\nOcean's Twe...,"Ocean's Twelve, the crime caper sequel starrin...",entertainment


In [83]:
# Load the spacy model
nlp = spacy.load("en_core_web_sm")

In [55]:
# Funktion für Tokenization, Remove stopwords, Lowercasing, Lemmatization, Remove punctuation 
def preprocess_text(text):
    # Tokenization
    doc = nlp(text)
    
    # Remove stopwords, Lowercasing, Lemmatization, Remove punctuation 
    preprocessed_tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    
    # Rückgabe der vorverarbeiteten Tokens als Text
    preprocessed_text = " ".join(preprocessed_tokens)
    return preprocessed_text

# Vorverarbeitung des Texts im DataFrame
df['Text'] = df['Text'].apply(preprocess_text)


In [25]:
#df = df.sample(n=500, random_state=1)

In [84]:
# Dictionary for converting the classes to numeric representation for the usage in the model
classes_mapping = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4
}

df = df.replace({"Class": classes_mapping})

In [86]:
# Balancing the dataset

# Zähle die Anzahl der Vorkommen jeder Kategorie
category_counts = df['Class'].value_counts()
print(category_counts)

# Bestimme die minimale Anzahl von Vorkommen einer Kategorie
min_count = min(category_counts)

balanced_dataset = pd.DataFrame()

# Iteriere über jede Kategorie
for category in category_counts.index:
    # Filtere die Zeilen des Datensatzes, die zur aktuellen Kategorie gehören
    category_subset = df[df['Class'] == category].sample(n=min_count, random_state=42)
    
    # Füge die ausgewählten Zeilen zur ausbalancierten Datenmenge hinzu
    balanced_dataset = balanced_dataset.append(category_subset)

# balanced_dataset enthält nun den ausbalancierten Datensatz

# Überprüfung der Kategorienverteilung im ausbalancierten Datensatz
balanced_category_counts = balanced_dataset['Class'].value_counts()
print(balanced_category_counts)

3    511
0    510
2    417
4    401
1    386
Name: Class, dtype: int64
3    386
0    386
2    386
4    386
1    386
Name: Class, dtype: int64


In [87]:
text = balanced_dataset.Text.values
classes = balanced_dataset.Class.values

In [88]:
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    # set number of classes to classes in df['Class']
    num_labels = len(df['Class'].unique()),
    output_attentions = False,
    output_hidden_states = False,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [89]:
token_id = []
attention_masks = []

def preprocessing(text, tokenizer):

  return tokenizer.encode_plus(
                        text,
                        add_special_tokens = True,
                        max_length = 128,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
classes = torch.tensor(classes)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [90]:
test_ratio = 0.2

batch_size = 32

# Indices of the train and test splits stratified by labels
train_idx, test_idx = train_test_split(
    np.arange(len(classes)),
    test_size = test_ratio,
    shuffle = True,
    stratify = classes)

# Train and test sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          classes[train_idx])

test_set = TensorDataset(token_id[test_idx], 
                        attention_masks[test_idx], 
                        classes[test_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

test_dataloader = DataLoader(
            test_set,
            sampler = SequentialSampler(test_set),
            batch_size = batch_size
        )

In [94]:
optimizer = torch.optim.AdamW(model.parameters(), 
                              # set learning rate
                              lr = 2e-5,
                            #   eps = 1e-08
                              )

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 2

for epoch in range(num_epochs):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    tr_accuracy = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    avg_loss = tr_loss / nb_tr_steps

    # ========== Evaluation ==========

    # Set model to evaluation mode
    model.eval()

    total_correct = 0
    total_samples = 0

    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            # Forward pass
            eval_output = model(b_input_ids, 
                            token_type_ids = None, 
                            attention_mask = b_input_mask)
            logits = eval_output.logits
            label_ids = b_labels.to('cpu').numpy()
            # Calculate validation metrics

            _, predicted_labels = torch.max(logits, 1)

            total_correct += (predicted_labels == b_labels).sum().item()
            total_samples += b_labels.size(0)

    accuracy = total_correct / total_samples

    print(f"Epoch {epoch+1}:")
    print(f"  Loss: {avg_loss:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")

Epoch 1:
  Loss: 0.0660
  Accuracy: 0.9741
Epoch 2:
  Loss: 0.0274
  Accuracy: 0.9896


In [93]:
import pickle
# save the model to disk
filename = './models/finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))